# prototyping

In [2]:
!pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=03bca6dab5f8c8c78d1ad577d16847287465a2a6f7d3c403490efe05078b27f9
  Stored in directory: /root/.cache/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k


In [20]:
import feedparser
import sqlite3
import time

# Replace the following variable with your own database name
DB_NAME = 'rss.db'
FEED_URL = 'https://www.forexlive.com/feed'

def create_entries_table():
    """
    Create the 'entries' table if it does not already exist
    """
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS entries
                 (title text, link text, published text)''')
    conn.commit()
    conn.close()

def store_feed_entry(entry):
    """
    Store a feed entry into the SQLite database
    """
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute('INSERT INTO entries (title, link, published) VALUES (?, ?, ?)', (entry.title, entry.link, entry.published))
    conn.commit()
    conn.close()

def check_new_entries():
    """
    Check for new feed entries and store them into the SQLite database
    """
    feed = feedparser.parse(FEED_URL)
    for entry in feed.entries:
        # Check if the entry is already in the database
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute('SELECT COUNT(*) FROM entries WHERE link = ?', (entry.link,))
        result = c.fetchone()
        conn.close()
        if result[0] == 0:
            # Store the new entry into the database
            store_feed_entry(entry)
            print(f"Stored new entry: {pubDate_str} - {title} - {description}...")

create_entries_table()

while True:
    check_new_entries()
    # Check for new entries every 10 seconds
    time.sleep(10)


Stored new entry: 2023-03-21 05:49:39 - ECB's Kazaks: It is not possible to say that we are done hiking rates - &lt;ul&gt;&lt;li&gt;Uncertainty is high in financial markets&lt;/li&gt;&lt;li&gt;But European banks are well capitalised&lt;/li&gt;&lt;li&gt;No reason to compare things with how it was in 2008&lt;/li&gt;&lt;li&gt;Bank supervision is much stronger now than it was back then&lt;/li&gt;&lt;/ul&gt;&lt;p style=&quot;&quot; class=&quot;text-align-justify&quot;&gt;The fact that they have gone from the message of rate hikes supposedly being a certainty to this kind of &quot;we cannot confirm nor deny&quot; sort of rhetoric speaks for itself.&lt;/p&gt;

                This article was written by Justin Low at www.forexlive.com....
Stored new entry: 2023-03-21 05:49:39 - ECB's Kazaks: It is not possible to say that we are done hiking rates - &lt;ul&gt;&lt;li&gt;Uncertainty is high in financial markets&lt;/li&gt;&lt;li&gt;But European banks are well capitalised&lt;/li&gt;&lt;li&gt;No re

KeyboardInterrupt: ignored

In [22]:
import feedparser
import sqlite3
from datetime import datetime

# establish database connection
conn = sqlite3.connect('rss_feed.db')
c = conn.cursor()

# create table if not exists
c.execute('''CREATE TABLE IF NOT EXISTS entries
             (title text, link text, pubDate text, creator text, category text, guid text, description text, comments text, canonical_link text)''')
conn.commit()

# retrieve and parse RSS feed
d = feedparser.parse("https://www.forexlive.com/feed")

# process feed entries
for entry in d.entries:
    # extract data from entry
    title = entry.title
    link = entry.link
    pubDate = datetime.strptime(entry.published, "%a, %d %b %Y %H:%M:%S %Z").strftime('%Y-%m-%d %H:%M:%S')
    creator = entry.get('author', '')
    category = entry.get('category', '')
    guid = entry.get('guid', '')
    description = entry.get('description', '')
    comments = entry.get('comments', '')
    canonical_link = entry.get('feedburner_origlink', '')

    # store data in database
    c.execute("INSERT INTO entries (title, link, pubDate, creator, category, guid, description, comments, canonical_link) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)",
              (title, link, pubDate, creator, category, guid, description, comments, canonical_link))
    conn.commit()

    # print confirmation message
    print("Stored new entry: " + pubDate + ", " + description)

# close database connection
conn.close()


Stored new entry: 2023-03-21 13:13:30, &lt;p&gt;One day before the FOMC, but the banking fears are abating. That has the EUR moving higher vs the USD, but the GBP is moving lower. The USDJPY is higher as expectations focus on the higher rates as some of the decline in yields is taken back.  What are the charts saying?&lt;/p&gt;

                This article was written by Greg Michalowski at www.forexlive.com.
Stored new entry: 2023-03-21 13:12:25, &lt;p&gt;US 2-year yields are up 23 bps to 4.15% today, which is back to where they were on Friday, though still 100 bps below where they were before the banking rout.&lt;/p&gt;&lt;p&gt;With so much volatility in short-term fund funds rate expectations, the 2-year is a good measure of where the market thinks the Fed will settle out. An average of 4.14% over the next two years might work out to 3.75-4.00% in Fed funds, with the usual added premium. &lt;/p&gt;&lt;p&gt;That idea makes intuitive sense if the Fed shifts to a strategy of keeping r

In [13]:
import feedparser
import sqlite3
from datetime import datetime

# set up the database
conn = sqlite3.connect('forex.db')
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS entries 
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              title TEXT, description TEXT, pubDate TEXT, category TEXT, link TEXT, guid TEXT)''')

# parse the feed and store new entries in the database
feed = feedparser.parse('https://www.forexlive.com/feed')
for entry in feed.entries:
    title = entry.title
    description = entry.description
    pubDate = entry.published_parsed
    pubDate_str = datetime.fromtimestamp(
        datetime.timestamp(datetime(*pubDate[:6]))).strftime('%Y-%m-%d %H:%M:%S')
    category = entry.category
    link = entry.link
    guid = entry.guid
    
    c.execute("SELECT * FROM entries WHERE guid=?", (guid,))
    if c.fetchone() is None:
        c.execute("INSERT INTO entries (title, description, pubDate, category, link, guid) VALUES (?, ?, ?, ?, ?, ?)",
                  (title, description, pubDate_str, category, link, guid))
        print(f"Stored new entry: {pubDate_str} - {title} - {description[:50]}...")
    else:
        c
    
conn.commit()
conn.close()


Entry already exists: 2023-03-21 12:15:28 - The EUR is the strongest and the NZD is the weakest as the North American session begins - &lt;p&gt;As the North American trading session com...
Entry already exists: 2023-03-21 12:07:20 - ForexLive European FX news wrap: Risk relief as banking turmoil abates for now - &lt;p&gt;Headlines:&lt;/p&gt;&lt;ul&gt;&lt;li&gt;&...
Entry already exists: 2023-03-21 11:26:31 - USD/CAD Technical Analysis - Will Fed Blink? - &lt;p&gt;On the daily chart below, we can
see that...
Entry already exists: 2023-03-21 11:06:00 - Yellen: Our actions show resolute commitment to ensure depositors' savings - &lt;ul&gt;&lt;li style=&quot;&quot; class=&quot;te...
Entry already exists: 2023-03-21 10:33:29 - The market continues to breathe a sigh of relief today - &lt;p style=&quot;&quot; class=&quot;text-align-ju...
Entry already exists: 2023-03-21 10:06:09 - AUD/USD Technical Analysis - &lt;p&gt;On the daily chart below, we can
see that...
Entry already exists: 2023-03-

In [12]:
import feedparser
import sqlite3
import re

# Connect to the database
conn = sqlite3.connect('rss_feed.db')

# Create a table for the posts if it doesn't already exist
conn.execute('''CREATE TABLE IF NOT EXISTS posts
             (title TEXT, pub_date TEXT, description TEXT, category TEXT, link TEXT, guid TEXT PRIMARY KEY)''')

# Parse the RSS feed
rss_url = 'https://www.forexlive.com/feed/all'
feed = feedparser.parse(rss_url)

# Loop through each post in the feed and insert it into the database
for post in feed.entries:
    title = re.sub(r'<.*?>', '', post.title)  # Remove HTML tags from the title
    description = re.sub(r'<.*?>', '', post.description)  # Remove HTML tags from the description
    pub_date = post.published_parsed
    pub_date = pub_date.strftime('%Y-%m-%d %H:%M:%S')
    category = post.category
    link = post.link
    guid = post.guid

    conn.execute(f"INSERT OR IGNORE INTO posts (title, pub_date, description, category, link, guid) VALUES (?, ?, ?, ?, ?, ?)", 
                (title, pub_date, description, category, link, guid))

# Commit the changes to the database and close the connection
conn.commit()
conn.close()

# Query the posts from the database and print them
conn = sqlite3.connect('rss_feed.db')
cursor = conn.execute("SELECT pub_date, title, description FROM posts")
for row in cursor:
    print(f"Date: {row[0]}, Title: {row[1]}, Description: {row[2]}")
conn.close()


# rss feedparser to SQLite

In [1]:
!pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 KB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=f154c8538dc5d76a5db3611200c740bcac5f98eb4169644efa2549d33e6111aa
  Stored in directory: /root/.cache/pip/wheels/65/7a/a7/78c287f64e401255dff4c13fdbc672fed5efbfd21c530114e1
Successfully built sgmllib3k


In [3]:
import feedparser
import sqlite3
import re
import time

# from bs4 import BeautifulSoup
# import sys
# if sys.version_info[0] >= 3:
#     unicode = str

# Connect to the database
conn = sqlite3.connect('rss.db')

# Create a table for the RSS feed items
conn.execute('''CREATE TABLE IF NOT EXISTS rss_items
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
              title TEXT NOT NULL,
              comments TEXT,
              pub_date TEXT,
              category TEXT,
              link TEXT NOT NULL,
              description TEXT NOT NULL,
              UNIQUE(link))''')



while True:
    # Parse the RSS feed
    feed = feedparser.parse('https://www.forexlive.com/feed')
    
    # Iterate through the RSS feed items
    for entry in feed.entries:
        # Check if the item already exists in the database
        existing_item = conn.execute(
            'SELECT * FROM rss_items WHERE link = ?', (entry.link,))
        if existing_item.fetchone():
            # Item already exists, skip to the next item
            continue
        
        # Item is new, extract the relevant fields and add them to the database
        title = re.sub('<.*?>', '', entry.title) # strip HTML tags from title
        comments = re.sub('<.*?>', '', entry.get('comments', '')) # strip HTML tags from comments (if any)
        pub_date = entry.get('published', '') # use published date if available
        if not pub_date:
            pub_date = entry.get('updated', '') # use updated date if published date is missing
        category = entry.get('tags', [{}])[0].get('term', '') # use first tag as category (if any)
        link = entry.link
        description = re.sub('<.*?>', '', entry.get('summary', '')) # strip HTML tags from description
        description = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', '', description)      
        # description = re.sub('&lt;', '', description)
        # description = re.sub('p&gt;', '', description)
        # description = re.sub('&gt;', '', description)
        # description = re.sub('ul&gt;', '', description)
        # description = re.sub('li&gt;', '', description)
        # description = re.sub('href=', '', description)
        # description = re.sub('target=', '', description)
        # description = re.sub('&quot;', '', description)
        # description = re.sub('a&gt;', '', description)
        # # description = re.sub('./', '. ', description)
        # # description = re.sub('.. ', '. ', description)
        # description = re.sub('rel=follow&gt;', '', description)
        # # description = re.sub('[/\s]+', ' ', description).replace('/', '').replace(' ', ' ')
        # description = re.sub('class=MsoNormal&gt;', '', description) 
        # description = re.sub('class=text-align-justify&gt;', '', description) 
        # description = re.sub(' p  ', '', description)
        # description = re.sub(' a  ', '', description)
        # description = re.sub(' P  ', '', description)
        # description = re.sub(' li  ', '', description)
        # description = re.sub('style=', '', description)
        # description = re.sub('https=', '', description)
        # description = re.sub('style:', '', description)
        # description = re.sub('https:', '', description)
        # description = re.sub('_blank', '', description)
        # description = re.sub('class=terms__main-term id=[a-zA-Z0-9-]+ &gt;', '', description)
        # description = re.sub('&apos;', "'", description)
        # description = re.sub(' li ul', '.', description)
        # description = re.sub('lili', '. ', description)
        # description = re.sub('rel=follow', '', description)
        # description = re.sub('class=text-align-justify', '', description)
        
        
        # Insert the new item into the database
        conn.execute('INSERT INTO rss_items (title, comments, pub_date, category, link, description) VALUES (?, ?, ?, ?, ?, ?)',
                     (title, comments, pub_date, category, link, description))
        
        # Print the new item
        print('New RSS item:')
        print('Title:', title)
        print('Pub. date:', pub_date)
        # print('Category:', category)
        print('Description:', description)
        print('-----------------------------------------------------------------')
        
    # Commit the changes to the database
    conn.commit()
    
    # Wait for 1 minute before checking again
    time.sleep(5)
    
# Close the database connection
conn.close()


New RSS item:
Title: Huge dislocations coming to the jobs market due to artificial intelligence - Goldman Sachs
Pub. date: Tue, 28 Mar 2023 18:24:20 GMT
Description: pThe impacts of artificial intelligence on society and politics are getting more attention but the macroeconomic impacts are just scratching the surface./ppGoldman Sachs chief economist Jan Hatzius write that generative AI, despite its uncertain potential, has shown remarkable progress in creating content similar to human-generated output and improving human-machine communication. He highlights the potential for  significant impacts on the labor force./pp“Despite significant uncertainty around the potential of generative AI, its ability to generate content that is indistinguishable from human-created output and to break down communication barriers between humans and machines reflects a major advancement with potentially large macroeconomic effects … We find that roughly two-thirds of US occupations are exposed to some degr

KeyboardInterrupt: ignored

# bs4

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://www.forexlive.com/feed"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()

# break into lines and remove leading and trailing space on each
lines = (line.strip() for line in text.splitlines())
# break multi-headlines into a line each
chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
# drop blank lines
text = '\n'.join(chunk for chunk in chunks if chunk)

print(text)

Forexlive RSS Breaking News Feed
https://www.forexlive.com/feed/
https://www.forexlive.com/
Real-time forex latest breaking news updates technical analysis, headlines FX charts, foreign exchange & currency trade from Forexlive.com
Thu, 23 Mar 2023 18:15:56 GMT
en-US
Copyright 2023 Forexlive.com
News
hourly
1
https://forexlive.s3.eu-west-1.amazonaws.com/fl-static/fl-logo-white.svg
Forexlive RSS Breaking News Feed
https://www.forexlive.com/
Gold nears $2000 once again. What's next
https://www.forexlive.com/news/gold-nears-2000-once-again-whats-next-20230323/#respond
Thu, 23 Mar 2023 18:01:38 GMT
Adam Button
News
https://www.forexlive.com/news/gold-nears-2000-once-again-whats-next-20230323/
https://www.forexlive.com/news/gold-nears-2000-once-again-whats-next-20230323/
https://www.forexlive.com/news/gold-nears-2000-once-again-whats-next-20230323/
www.forexlive.com
&lt;p&gt;Gold prices briefly traded above $2000 on Monday but quickly turned around and closed below the key level before falli

/usr/local/lib/python3.9/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
